# How to build Agents with Couchbase, LangChain and Letta!

This repository will illustrate the core concepts behind [Letta](https://www.letta.com/), a new startup built from the success of [MemGPT](https://memgpt.ai/) from researchers at UC Berkeley.

Letta helps you build Agents with **meta memory**.

In addition to the prompt sent to an LLM, the LLM will also be provided with a short description of its internal knowledge about the user and their interests derived from previous chats.

There are 3 parts to this notebook:

## 1. Getting Started with Letta Agents

An illustration of the basic setup requirements and a simple example of an Agent with internal memory.

## 2. Upload Couchbase Search Tool to Letta

Wrap Couchbase Vector Search in a Python function and upload it to the Letta Tool server. We will use [LangChain](https://python.langchain.com/docs/integrations/providers/couchbase/) to create function which uses Couchbase Vector Store. This tool focuses on providing RAG capabilities for a particular collection. We can create either multiple such functions, or a single function with collection name passed as parameter to offer varied results based on use case.

## 3. Letta Agent with Couchbase

Construct a Letta Agent that chats with a climate report indexed in Couchbase. Observe how the Letta Agent develops an internal model of the chat user and the climate report through these interactions.

# 1. Getting Started with Letta Agents

### Primer: Install Letta Server

`$pip install -U letta`

Start local server:
(You may need to export OpenAI API Key before running this command for this tutorial to work)

`$letta server`



For more details, checkout ReadMe of the Letta Repo
https://github.com/letta-ai/letta



### Connect to Letta Client

Connect the Letta Server, the default port to run letta client is 8283. This may change based on your configuration.

In [24]:
from letta import create_client 

# connect to the letta server
letta_client = create_client(base_url="http://localhost:8283")



In [43]:
# get this from your local GUI
my_letta_agent_id = "agent-ba1cee1d-1f31-4619-8ee2-557eb844a063"

# send a message to the agent
response = letta_client.send_message(
    agent_id=my_letta_agent_id,
    role="user", 
    message="By 2100, what is the projected global warming rise possible if no actions are taken?"
)

In [42]:
# Print the internal monologue
print("Internal Monologue:")
print(response.messages[0].internal_monologue)
print()

# Print the message sent
print("Message Sent:")
print(response.messages[1].function_call.arguments)
print()

# Print the function return status
print("Function Return Status:")
print(f"Status: {response.messages[2].status}")
print(f"Time: {response.messages[2].function_return}")
print()

# Print usage statistics
print("Usage Statistics:")
print(f"Completion tokens: {response.usage.completion_tokens}")
print(f"Prompt tokens: {response.usage.prompt_tokens}") 
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Step count: {response.usage.step_count}")

Internal Monologue:
His concern about global climate trends is apparent. While providing the information, I must stress the gravity and urgency of environmental preservation.

Message Sent:
{
  "message": "This is indeed an issue of utmost importance, Chad. If no substantial actions are taken to mitigate climate change, scientists predict that global temperatures could increase by more than 4 degrees Celsius by the end of the 21st century compared to pre-industrial levels. However, it's important to note that projections can vary based on different climate models and scenarios. The current consensus underscores the need for immediate and sustained action to limit warming to within manageable levels."
}

Function Return Status:
Status: success
Time: {
  "status": "OK",
  "message": "None",
  "time": "2024-11-15 10:23:55 AM IST+0530"
}

Usage Statistics:
Completion tokens: 136
Prompt tokens: 6199
Total tokens: 6335
Step count: 1


# 2. Upload Couchbase Search Tool indexed on Climate Report to Letta

We are using a publically available IPCC Climate Change Report. You may get it from here: https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_LongerReport.pdf .

To learn how to index a PDF to Couchbase to perform vector search, please follow this tutorial, https://developer.couchbase.com/tutorial-python-langchain-pdf-chat


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-xxxxxxx"
os.environ["CB_CONN_STR"] = "couchbase://localhost"
os.environ["CB_BUCKET"] = "pdf-docs"
os.environ["CB_SCOPE"] = "letta"
os.environ["CB_COLLECTION"] = "climate_report"
os.environ["CB_USERNAME"] = "Administrator"
os.environ["CB_PASSWORD"] = "password"
os.environ["SEARCH_INDEX"] = "climate_index"

In [45]:
def ask_climate_search(
    self,
    search_query: str
):
    """
    Search the couchbase database for all the climate related questions from climate report.
    It uses the search engine to find the most relevant results related to climate and impacts.
    All the climate related questions shoukld be asked using this function.

    Args: 
        search_query (str): The search query for the climate related questions.

    Returns: 
        search_results (str): The results from the search engine.
    """
    from couchbase.cluster import Cluster, ClusterOptions
    from couchbase.auth import PasswordAuthenticator
    from datetime import timedelta
    from langchain_openai import OpenAIEmbeddings
    from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore
    import os
    from pydantic import SecretStr


    conn_str = os.getenv("CB_CONN_STR") or "couchbase://localhost"
    bucket_name = os.getenv("CB_BUCKET") or "pdf-docs"
    scope_name = os.getenv("CB_SCOPE") or "letta"
    collection_name = os.getenv("CB_COLLECTION") or "climate_report"
    username = os.getenv("CB_USERNAME") or "Administrator"
    password = os.getenv("CB_PASSWORD") or "password"
    search_index = os.getenv("SEARCH_INDEX") or "climate_index"
    openai_api_key = os.getenv("OPENAI_API_KEY") or "sk-xxxxxxx"

    # Connect to the cluster
    auth = PasswordAuthenticator(username, password)
    options = ClusterOptions(auth)
    cluster = Cluster(conn_str, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    embedding = OpenAIEmbeddings(api_key=SecretStr(openai_api_key))
    cb_vector_store = CouchbaseSearchVectorStore(
        cluster,
        bucket_name,
        scope_name,
        collection_name,
        embedding,
        search_index,
    )

    query_result = cb_vector_store.similarity_search(search_query, 5)
    formatted_results = "\n".join(
        [f"[Search Result {i+1}] {str(result.page_content)}" for i, result in enumerate(query_result)]
    )
    return formatted_results

In [46]:
couchbase_search_tool = letta_client.create_tool(
    ask_climate_search,
    tags=["search", "climate", "weather",  "impact"],
)

In [ ]:
letta_client.list_tools()

# 3. Letta Agent with Couchbase

In [ ]:
from letta import LLMConfig, EmbeddingConfig
# set default llm config for agents
letta_client.set_default_llm_config(
    LLMConfig.default_config(model_name="gpt-4o-mini")
)

# set default embedding config for agents
letta_client.set_default_embedding_config(
    EmbeddingConfig.default_config(model_name="text-embedding-ada-002")
)

agent_state = letta_client.create_agent(
    name="climate-agent",
    tools=[couchbase_search_tool.name],
)

new_agent_id = agent_state.id

In [38]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="By 2100, what is the projected global warming rise possible if no actions are taken?"
)

print(response)

{
    "messages": [
        {
            "id": "message-b8555891-125f-4cb6-9f18-b55c0f7ff246",
            "date": "2024-11-15T04:41:41+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "The user is curious about the implications of climate inaction. I should provide a well-researched answer about global warming projections."
        },
        {
            "id": "message-b8555891-125f-4cb6-9f18-b55c0f7ff246",
            "date": "2024-11-15T04:41:41+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "ask_climate_search",
                "arguments": "{\n  \"search_query\": \"global warming rise by 2100\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_e9JFmCstpfODePsSUaOf8tZA"
            }
        },
        {
            "id": "message-bf02e7e0-3fe7-4f2e-8bc2-2ce5df10e7d0",
            "date": "2024-11-15T04:41:41+00:00",
            "message_type": "fu

In [39]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="Based on the report, Does greenhouse gases emission depend on income level?"
)

print(response)

{
    "messages": [
        {
            "id": "message-225864f8-b60e-4202-b61b-7cd2b88c0a9c",
            "date": "2024-11-15T04:41:45+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "Chad is asking an insightful question about the relationship between income levels and emissions. I need to find relevant information."
        },
        {
            "id": "message-225864f8-b60e-4202-b61b-7cd2b88c0a9c",
            "date": "2024-11-15T04:41:45+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "ask_climate_search",
                "arguments": "{\n  \"search_query\": \"greenhouse gas emissions and income level\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_fU0IqHN9balZ2nfgnROyXtAn"
            }
        },
        {
            "id": "message-dda36b14-49b9-485b-865d-a7d5ed8e186f",
            "date": "2024-11-15T04:41:46+00:00",
            "message_t

In [40]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="What are some high confidence methods which can help in reversing climate change?"
)

print(response)

{
    "messages": [
        {
            "id": "message-8ccb5cbb-2e22-47a2-86a5-fa10e3554e1e",
            "date": "2024-11-15T04:41:50+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "Chad is looking for effective methods to combat climate change. I need to gather reliable information about high-confidence strategies."
        },
        {
            "id": "message-8ccb5cbb-2e22-47a2-86a5-fa10e3554e1e",
            "date": "2024-11-15T04:41:50+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "ask_climate_search",
                "arguments": "{\n  \"search_query\": \"methods to reverse climate change\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_c1XF43X8Aazdaa818uAQlHQw"
            }
        },
        {
            "id": "message-865a92bd-ae87-408f-bfea-3788f74dfba6",
            "date": "2024-11-15T04:41:51+00:00",
            "message_type": "